In [3]:
from PIL import Image
import matplotlib.pyplot as plt
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
from torch import optim 
import skillsnetwork 

In [ ]:
await skillsnetwork.prepare("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip", path = "/home/bruno/DeepLearning_PyTorch", overwrite=True)

In [4]:
class Dataset(Dataset):

    # Constructor
    def __init__(self,transform=None,train=True):
        directory="/home/bruno/DeepLearning_PyTorch"
        positive="Positive"
        negative="Negative"

        positive_file_path=os.path.join(directory,positive)
        negative_file_path=os.path.join(directory,negative)
        positive_files=[os.path.join(positive_file_path,file) for file in  os.listdir(positive_file_path) if file.endswith(".jpg")]
        positive_files.sort()
        negative_files=[os.path.join(negative_file_path,file) for file in  os.listdir(negative_file_path) if file.endswith(".jpg")]
        negative_files.sort()
        number_of_samples=len(positive_files)+len(negative_files)
        self.all_files=[None]*number_of_samples
        self.all_files[::2]=positive_files
        self.all_files[1::2]=negative_files 
        # The transform is goint to be used on image
        self.transform = transform
        #torch.LongTensor
        self.Y=torch.zeros([number_of_samples]).type(torch.LongTensor)
        self.Y[::2]=1
        self.Y[1::2]=0
        
        if train:
            self.all_files=self.all_files[0:10000] #Change to 30000 to use the full test dataset
            self.Y=self.Y[0:10000] #Change to 30000 to use the full test dataset
            self.len=len(self.all_files)
        else:
            self.all_files=self.all_files[30000:]
            self.Y=self.Y[30000:]
            self.len=len(self.all_files)    
       
    # Get the length
    def __len__(self):
        return self.len
    
    # Getter
    def __getitem__(self, idx):
        
        
        image=Image.open(self.all_files[idx])
        y=self.Y[idx]
          
        
        # If there is any transform method, apply it onto the image
        if self.transform:
            image = self.transform(image)

        return image, y

In [5]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
# transforms.ToTensor()
#transforms.Normalize(mean, std)
#transforms.Compose([])

transform =transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

In [6]:
dataset_train=Dataset(transform=transform,train=True)
dataset_val=Dataset(transform=transform,train=False)

In [7]:
dataset_train[0][0].shape

torch.Size([3, 227, 227])

In [8]:
size_of_image=3*227*227
size_of_image

154587

In [9]:
torch.manual_seed(0)

In [10]:
class SoftMax(nn.Module):
    def __init__(self, number_inputs, number_outputs=2):
        super(SoftMax, self).__init__()
        self.linear = nn.Linear(number_inputs, 256)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(256, 2)
    
    def forward(self, x):
        x = self.relu(self.linear(x))
        x = self.linear2(x)
        return x

In [11]:
model = SoftMax(number_inputs=size_of_image, number_outputs=2)

In [12]:
torch.cuda.is_available()
device = torch.device('cuda:0')
model.to(device)

SoftMax(
  (linear): Linear(in_features=154587, out_features=256, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=256, out_features=2, bias=True)
)

In [13]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.1)

In [14]:
criterion = nn.CrossEntropyLoss()

In [15]:
data_train = DataLoader(dataset_train, batch_size = 5)
data_val = DataLoader(dataset_val, batch_size = 5)

In [ ]:
epochs = 10
LOSS_TRAIN = []
correct = 0
ACCURACY = []
print("Training has begun\n")
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for x, y in data_train:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        model_train = model(x.view(-1, size_of_image))
        loss = criterion(model_train, y)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    loss_train = running_loss / len(dataset_train)
    LOSS_TRAIN.append(loss_train)
    
    with torch.no_grad():
        model.eval()
        correct = 0
        count = 0
        for x_val, y_val in data_val:
            x_val, y_val = x_val.to(device), y_val.to(device)
            #print(y_val)
            model_val = model(x_val.view(-1, size_of_image))
            value, prediction = torch.max(model_val, 1)
            correct += (prediction==y_val).sum().item()
            count += 1
            
        #print(count)  
        accuracy = correct / len(dataset_val)

    # with torch.no_grad():
    #     model.eval()
    #     correct = 0
    #     count = 0
    #     for x_train, y_train in data_train:
    #         x_train, y_train = x_train.to(device), y_train.to(device)
    #         #print(y_val)
    #         model_test = model(x_train.view(-1, size_of_image))
    #         value, prediction = torch.max(model_test, 1)
    #         correct += (prediction==y_train).sum().item()
    #         count += 1
            
    #     #print(count)  
    #     accuracy_train = correct / len(dataset_train)
        
    ACCURACY.append(accuracy)
    
    print("Epoch: ", epoch + 1, "Train Loss: ", loss_train, "Accuracy Training Data: ", "Accuracy: ", accuracy)

In [ ]:
prediction

In [ ]:
y_val

In [ ]:
print((prediction==y_val).sum().item())

In [ ]:
len(dataset_val)